In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
#from langchain_openai import ChatOpenAI
import operator
from typing import Annotated, TypedDict, Union
from langchain_core.agents import AgentAction, AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor
from langchain import hub

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_openai.chat_models import ChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.agents import create_react_agent
from typing import List, Sequence
import os
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph
from dotenv import load_dotenv
load_dotenv()

False

In [17]:
import pandas as pd

# creating the Chains

In [3]:

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet."
            "Always provide detailed recommendations, including requests for length, virality, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
            " Generate the best twitter post possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

print('executed this far')

executed this far


# creating the nodes

In [7]:
llm = ChatOpenAI()
#llm = AzureOpenAI(openai_api_version = 'turbo-2024-04-09')
generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

REFLECT = "reflect"
GENERATE = "generate"


def generation_node(state: Sequence[BaseMessage]):
    return generate_chain.invoke({"messages": state})


def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": messages})
    return [HumanMessage(content=res.content)]




# Build the Graph

In [9]:
builder = MessageGraph()
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(GENERATE)

def should_continue(state: List[BaseMessage]):
    if len(state) > 6: # continue 
        return END
    return REFLECT


builder.add_conditional_edges(GENERATE, should_continue)
builder.add_edge(REFLECT, GENERATE)

graph = builder.compile()
print(graph.get_graph().draw_mermaid())




%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	generate(generate)
	reflect(reflect)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate;
	reflect --> generate;
	generate -.-> reflect;
	generate -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [10]:
graph.get_graph().print_ascii()

          +-----------+            
          | __start__ |            
          +-----------+            
                *                  
                *                  
                *                  
          +----------+             
          | generate |             
          +----------+             
          ...        ...           
         .              .          
       ..                ..        
+---------+           +---------+  
| reflect |           | __end__ |  
+---------+           +---------+  


In [11]:
if __name__ == "__main__":
    print("Hello LangGraph")
    inputs = HumanMessage(content="""Make this tweet better:"
                                    @LangChainAI
            — newly Tool Calling feature is seriously underrated.

            After a long wait, it's  here- making the implementation of agents across different models with function calling - super easy.

            Made a video covering their newest blog post

                                  """)
    response = graph.invoke(inputs)
print(response)

Hello LangGraph
[HumanMessage(content='Make this tweet better:"\n                                    @LangChainAI\n            — newly Tool Calling feature is seriously underrated.\n\n            After a long wait, it\'s  here- making the implementation of agents across different models with function calling - super easy.\n\n            Made a video covering their newest blog post\n\n                                  ', additional_kwargs={}, response_metadata={}, id='ce72890d-c737-4f2e-9645-cdedaa9da344'), AIMessage(content='"🚀 Excited to share that @LangChainAI has rolled out a game-changing Tool Calling feature! 🛠️✨ Say goodbye to complex agent implementation across models - it\'s now a breeze. Check out my video unpacking their latest blog post! #AI #Tech #Innovation"', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 113, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens

In [12]:
response[0]

HumanMessage(content='Make this tweet better:"\n                                    @LangChainAI\n            — newly Tool Calling feature is seriously underrated.\n\n            After a long wait, it\'s  here- making the implementation of agents across different models with function calling - super easy.\n\n            Made a video covering their newest blog post\n\n                                  ', additional_kwargs={}, response_metadata={}, id='ce72890d-c737-4f2e-9645-cdedaa9da344')

In [13]:
response[1]

AIMessage(content='"🚀 Excited to share that @LangChainAI has rolled out a game-changing Tool Calling feature! 🛠️✨ Say goodbye to complex agent implementation across models - it\'s now a breeze. Check out my video unpacking their latest blog post! #AI #Tech #Innovation"', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 113, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-49ce3c0b-55aa-4d78-b707-c8a4dfa299c2-0', usage_metadata={'input_tokens': 113, 'output_tokens': 63, 'total_tokens': 176, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [15]:
response[-2]

HumanMessage(content='This revised tweet effectively captures attention with engaging language, highlights the key benefits of the Tool Calling feature, and encourages followers to watch the video for more insights. To further enhance the tweet, consider adding a call-to-action prompting followers to share their thoughts or experiences with similar features. Additionally, incorporating a teaser or a surprising fact from the blog post can create anticipation and prompt curiosity. Including a visual element like a screenshot or a snippet from the video can also increase visual appeal and engagement. Lastly, ensure the tweet is posted during peak engagement times to reach a wider audience and maximize virality.', additional_kwargs={}, response_metadata={}, id='31ff89f2-31fa-4719-ab14-2f2cecd4e938')

In [14]:
response[-1]

AIMessage(content='"🌟 Dive into the future of AI with @LangChainAI\'s groundbreaking Tool Calling feature! Simplifying agent integration like never before. Watch my video for an exclusive peek at how this innovation is reshaping the tech landscape. Share your thoughts on revolutionizing AI models! #AI #LangChainAI #Innovation #TechRevolution 🚀"', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 703, 'total_tokens': 776, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6a8bbc2c-96e1-447f-ba7f-b929046d869d-0', usage_metadata={'input_tokens': 703, 'output_tokens': 73, 'total_tokens': 776, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d

In [26]:
df = pd.DataFrame(response)
pd.options.display.max_colwidth = 100
for value in df[0].values :
    print( value)
    print('----------------------------------------------> ')

('content', 'Make this tweet better:"\n                                    @LangChainAI\n            — newly Tool Calling feature is seriously underrated.\n\n            After a long wait, it\'s  here- making the implementation of agents across different models with function calling - super easy.\n\n            Made a video covering their newest blog post\n\n                                  ')
----------------------------------------------> 
('content', '"🚀 Excited to share that @LangChainAI has rolled out a game-changing Tool Calling feature! 🛠️✨ Say goodbye to complex agent implementation across models - it\'s now a breeze. Check out my video unpacking their latest blog post! #AI #Tech #Innovation"')
----------------------------------------------> 
('content', 'This revised tweet captures attention with emojis, highlights the key points concisely, and includes relevant hashtags to increase visibility. To enhance it further, consider adding more specific details about the benefits of